In [56]:
import pandas as pd
import os
import datetime

In [57]:
def sort_categories(security):
    if 'Call' in security:
        return 'call'
    elif 'Put' in security:
        return 'put'
    else:
        return 'stock'
    
def get_symbol(security, category):
    if category=='stock':
        return security.split()[-1][1:-1]
    else:
        return security
    
def get_underlying(symbol):
    try:
        underlying = symbol.split()[0]
        return underlying
    except BaseException:
        return
    
def get_hold_duration(open_date, close_date):
    t0 = datetime.datetime.strptime(open_date, '%Y-%m-%d %H:%M:%S')
    t1 = datetime.datetime.strptime(close_date, '%Y-%m-%d %H:%M:%S')
    return (t1-t0)

In [58]:
data_path = './data'
files = ['RC_869434357_20150101_20151231.xlsx', 'RC_869434357_20160101_20161231.xlsx', 'RC_869434357_20170101_20171231.xlsx',
         'RC_869434357_20180101_20181231.xlsx','RC_869434357_20190101_20191231.xlsx', 'RC_869434357_20200101_20201231.xlsx', 
         'RC_869434357_20210101_20210501.xlsx']   

In [59]:
data = []
for i in files[:3]:
    file_path = os.path.join(data_path, i)
    data.append(pd.read_excel(file_path, dtype='str')[:-1])

/Users/macuser/anaconda3/envs/p38/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [60]:
df = pd.concat(data)

In [61]:
df.columns = [i.lower().replace(' ', '_') for i in df.columns]
# df = df[df['trans_type'] != 'Wash Sale Adj']
# df = df[df['trans_type'] != 'Short WS Adj']
df.head()

,security,trans_type,qty,open_date,adj_cost,close_date,adj_proceeds,gain_adj,adj_gain($),adj_gain(%),term
0,APPLE INC (AAPL),Sell.FIFO,28,2015-10-02 00:00:00,3043.51,2015-10-09 00:00:00,3128.75,NaN,85.24,2.8,Short-term
1,ADOBE INC (ADBE),Sell.FIFO,56,2015-12-09 00:00:00,5015.84,2015-12-17 00:00:00,5305.55,NaN,289.71,5.78,Short-term
2,ADOBE INC (ADBE),Sell.FIFO,45,2015-12-10 00:00:00,3997.17,2015-12-17 00:00:00,4263.38,NaN,266.21,6.66,Short-term
3,ANALOG DEVICES INC (ADI),Sell.FIFO,45,2015-11-23 00:00:00,2569.23,2015-11-24 00:00:00,2651.04,NaN,81.81,3.18,Short-term
4,ANALOG DEVICES INC (ADI),Sell.FIFO,45,2015-11-23 00:00:00,2569.23,2015-11-24 00:00:00,2640.03,NaN,70.8,2.76,Short-term


In [62]:
df.shape

(1163, 11)

In [63]:
df['category'] = df['security'].apply(lambda x: sort_categories(x))
df['symbol'] = df.apply(lambda x: get_symbol(x.security, x.category), axis=1)
df['underlying'] = df['symbol'].apply(lambda x: get_underlying(x))
df['hold_duration'] = df.apply(lambda x: get_hold_duration(x.open_date, x.close_date), axis=1)
df['hold_duration'] = df.apply(lambda x: datetime.timedelta(0) if x.trans_type=='Wash Sale Adj' else x.hold_duration, axis=1)

In [64]:
df.to_excel('./data/performance.xlsx', index=False)